In [52]:
from pulp import *
import pandas as pd
from collections import defaultdict

In [53]:
# Cargar datos
df_vehiculos = pd.read_excel('../../Datos_P1/df_vehicle.xlsx')
df_clientes = pd.read_excel('../../Datos_P1/df_orders.xlsx')
df_distancias = pd.read_excel('../../Datos_P1/df_distance_km.xlsx')

df_distancias.index = df_distancias.columns


In [54]:
# Preparación de datos
capacidades = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["capacidad_kg"]))
costos = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["costo_km"]))
autonomias = dict(zip(df_vehiculos["vehiculo_id"], df_vehiculos["autonomia_km"]))

pedidos = dict(zip(df_clientes["cliente"], df_clientes["order_demand"]))
pedidos["Almacén"] = 0  # Asegurar que el almacén no tenga demanda

distancias = df_distancias.stack().to_dict()

clientes = df_clientes["cliente"].tolist() + ["Almacén"]
vehiculos = df_vehiculos["vehiculo_id"].tolist()

# Evitar distancias de 0 entre clientes
# for i in clientes:
#     for j in clientes:
#         if i != j and (i, j) in distancias and distancias[(i, j)] == 0:
#             distancias[(i, j)] = float('inf')  # Penalizar distancias 0 para evitar bucles

In [55]:
# Crear el problema
problema = LpProblem("Ruteo_de_Vehiculos", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(i, j, k) for i in clientes for j in clientes for k in vehiculos if i != j], cat="Binary")
q = LpVariable.dicts("q", [(i, k) for i in clientes for k in vehiculos], lowBound=0)
u = LpVariable.dicts("u", [(i, k) for i in clientes for k in vehiculos], lowBound=0)

# Función objetivo: minimizar costo total
problema += lpSum(x[i, j, k] * costos[k] * distancias.get((i, j), 0) for i in clientes for j in clientes for k in vehiculos if i != j)

# Restricciones

# Cada cliente debe ser atendido exactamente una vez
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(x[i, j, k] for j in clientes for k in vehiculos if i != j) == 1

# Cada vehículo no debe exceder su capacidad en cada viaje
for k in vehiculos:
    problema += lpSum(q[i, k] for i in clientes) <= capacidades[k]

# Satisfacer la demanda de los clientes
for i in clientes:
    if i != 'Almacén':
        problema += lpSum(q[i, k] for k in vehiculos) == pedidos[i]

# Flujo continuo: entrada = salida para evitar bucles
for k in vehiculos:
    for i in clientes:
        if i != 'Almacén':
            problema += lpSum(x[i, j, k] for j in clientes if j != i) == lpSum(x[j, i, k] for j in clientes if j != i)

# Permitir múltiples viajes al almacén para recargar capacidad y continuar con la ruta
for k in vehiculos:
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') >= 1
    problema += lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén') >= 1
    problema += lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén') == lpSum(x[i, 'Almacén', k] for i in clientes if i != 'Almacén')
    problema += lpSum(x[i, j, k] for i in clientes for j in clientes if i != j) >= lpSum(x['Almacén', j, k] for j in clientes if j != 'Almacén')

# Restringir autonomía del vehículo
for k in vehiculos:
    problema += lpSum(x[i, j, k] * distancias.get((i, j), 0) for i in clientes for j in clientes if i != j) <= autonomias[k]

# Prohibir rutas cíclicas directas
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if i != j:
                problema += x[i, j, k] + x[j, i, k] <= 1

# Resolver el problema
solver = PULP_CBC_CMD(msg=1, timeLimit=300)
problema.solve(solver)

# Imprimir el estado de la solución
print(LpStatus[problema.status])

Optimal


In [56]:
# Imprimir rutas resultantes con costos y capacidades
costo_total_sistema = 0
for k in vehiculos:
    print(f"\nVehículo {k}:")
    ruta = []
    carga_total = 0
    distancia_total = 0
    costo_total_vehiculo = 0
    
    for i in clientes:
        for j in clientes:
            if i != j and x[i, j, k].value() == 1:
                ruta.append((i, j))
                distancia_total += distancias.get((i, j), 0)
                costo_total_vehiculo += costos[k] * distancias.get((i, j), 0)
    
    for i in clientes:
        if q[i, k].value() > 0:
            carga_total += q[i, k].value()
    
    for segmento in ruta:
        print(f"  Va de {segmento[0]} a {segmento[1]}")
    print(f"  Carga total: {carga_total} kg")
    print(f"  Distancia total: {distancia_total} km")
    print(f"  Costo total vehículo: {costo_total_vehiculo}")
    
    costo_total_sistema += costo_total_vehiculo

# Detectar clientes no visitados
clientes_visitados = set()
for k in vehiculos:
    for i in clientes:
        for j in clientes:
            if i != j and x[i, j, k].value() == 1:
                clientes_visitados.add(j)

clientes_no_visitados = set(clientes) - clientes_visitados - {"Almacén"}
print(f"\nClientes no visitados: {clientes_no_visitados}")

# Imprimir el costo total final
print(f"\nCosto total del sistema: {costo_total_sistema}")


Vehículo 1:
  Va de Cliente_4 a Almacén
  Va de Cliente_6 a Cliente_4
  Va de Almacén a Cliente_6
  Carga total: 2026.0 kg
  Distancia total: 6.9744 km
  Costo total vehículo: 1.3948800000000001

Vehículo 2:
  Va de Cliente_14 a Cliente_17
  Va de Cliente_17 a Almacén
  Va de Almacén a Cliente_14
  Carga total: 4362.0 kg
  Distancia total: 12.5527 km
  Costo total vehículo: 1.757378

Vehículo 3:
  Va de Cliente_1 a Cliente_11
  Va de Cliente_11 a Almacén
  Va de Almacén a Cliente_1
  Carga total: 4881.0 kg
  Distancia total: 8.709999999999999 km
  Costo total vehículo: 1.742

Vehículo 4:
  Va de Cliente_10 a Almacén
  Va de Cliente_16 a Cliente_10
  Va de Almacén a Cliente_16
  Carga total: 3321.0 kg
  Distancia total: 10.107 km
  Costo total vehículo: 1.9203299999999999

Vehículo 5:
  Va de Cliente_5 a Cliente_9
  Va de Cliente_9 a Almacén
  Va de Almacén a Cliente_5
  Carga total: 4148.0 kg
  Distancia total: 5.5911 km
  Costo total vehículo: 1.789152

Vehículo 6:
  Va de Cliente_2 